In [1]:
import numpy as np
import matplotlib.pyplot as plt
import qutip as qt

# Grovers Algorithm 
## Introduction

Time complexity is a representation of the time taken by an algorithm to solve a problem as a function relative to the size of the input data. This property is commonly expressed in $\mathcal{O}$ ("Big O") notation which represents the upper bound of the number of steps needed to solve the problem.

If you have $N$ randomly ordered elements and you need to find a particular element, in classical computing the time complexity is $\mathcal{O}(N)$.

In the context of quantum computing, Grover's algorithm seeks to find a particular element with $\mathcal{O}(\sqrt{N})$ time complexity. The decreased time complexity is due to the superposition of states, which allows for simultaneous examination of multiple elements.

This report aims to examine Grovers Algorithm and create an executable demo using 'qutip' (Quantum Toolbox in Python)

by Jessie Lonsdale and Anastasios Mavridis.

In [15]:
# Define the size of the search space and the target state
n = 2  # number of qubits
N = 2**n  # size of the search space
target_state = qt.tensor(qt.basis(2,1), qt.basis(2,1))  # the target state we want to find

# Create the quantum gates needed for Grover's algorithm
hadamard = qt.hadamard_transform(n)
oracle = qt.qdiags([1 if i == 3 else -1 for i in range(N)], 0)
diffusion = 2 * qt.ket2dm(qt.tensor(qt.basis(2,1), qt.basis(2,1))) - np.eye(N)

# Define the number of iterations of the algorithm
num_iterations = int(np.pi / 4 * np.sqrt(N))

# Apply Grover's algorithm
state = qt.tensor(hadamard, qt.basis(N, 0))
for i in range(num_iterations):
    state = diffusion * oracle * state

# Measure the final state
measurement = qt.measure(state.ptrace(0), qt.basis(N, 0))
print("Measured state:", measurement)



C:\Users\lonsd\AppData\Local\Temp\ipykernel_5508\3418106330.py:7: DeprecationWarning: Importing functions/classes of the qip submodule directly from the namespace qutip is deprecated. Please import them from the submodule instead, e.g.
from qutip.qip.operations import cnot
from qutip.qip.circuit import QubitCircuit

  hadamard = qt.hadamard_transform(n)


TypeError: Incompatible quantum object dimensions